In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
cudnn.benchmark = True

In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize(256),
        #transforms.RandomRotation(45, fill=255), # https://pytorch.org/vision/stable/generated/torchvision.transforms.RandomRotation.html
        transforms.CenterCrop(224),
        #transforms.functional.equalize,  # https://pytorch.org/vision/main/generated/torchvision.transforms.functional.equalize.html
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3,1,1)),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        #transforms.functional.equalize, 
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3,1,1)),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../input/fingerprintbinarywithunknown'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}


dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# Load evaluation data
eval_data_dir = '../input/eval-data'

eval_datasets = {x: datasets.ImageFolder(os.path.join(eval_data_dir, x),
                                          data_transforms['val'])
                  for x in ['live', 'fake_all', 'fake_known', 'fake_unknown']}
                 

eval_dataloaders = {x: torch.utils.data.DataLoader(eval_datasets[x], batch_size=1,
                                             shuffle=False, num_workers=0)
                 for x in ['live','fake_all', 'fake_known','fake_unknown']}

eval_dataset_sizes = {x: len(eval_datasets[x]) for x in ['live', 'fake_all', 'fake_known', 'fake_unknown']}

In [6]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

#  Get a batch of training data
eval_inputs, eval_classes = next(iter(eval_dataloaders['live']))
# Make a grid from batch
eval_out = torchvision.utils.make_grid(eval_inputs)
imshow(eval_out, title=[class_names[x] for x in eval_classes+1])

In [7]:
def initialize_model(model_name, num_classes, train_last_layer_only, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    #input_size = 224 is assumed as all pre-trained models are using 224x244 dimesions

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, train_last_layer_only)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        if use_pretrained:
            optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.000001) 
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
        else:
            optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.000001) 
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
            
    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, train_last_layer_only)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        if use_pretrained:
            optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0000001) 
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1) 
        else:
            optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.000001) 
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)

    elif model_name == "vgg16":
        """ VGG16
        """
        model_ft = models.vgg16(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, train_last_layer_only)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        if use_pretrained:
            optimizer_ft = optim.Adam(model_ft.parameters(), 0.0000001) #V8
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
        else:
            optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.000001) 
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, train_last_layer_only)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        if use_pretrained: # tested with 000001
            optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.000001)
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
        else:
            optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.000001) 
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, train_last_layer_only)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        if use_pretrained:
            optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.00001)
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
        else:
            optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001) 
            exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, optimizer_ft, exp_lr_scheduler

In [8]:
def set_parameter_requires_grad(model, train_last_layer_only=False):
    if train_last_layer_only:
        for param in model.parameters():
            param.requires_grad = False

In [9]:
def train_all(model_names, epochs=30):
    for model_name in model_names:
        
        # train all alyers with ImgNet initialized weights 
        print("\nUsing model " + model_name + " pre-trained and training all layers")
        print("Starting training")
        model_ft, optimizer_ft, exp_lr_scheduler = initialize_model(model_name, len(class_names), train_last_layer_only=False, use_pretrained=True)
        model_ft = model_ft.to(device)
        criterion = nn.CrossEntropyLoss()
        best_model, train_lss, val_lss, train_acc,val_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=epochs)
        acc = evaluate_model(best_model, eval_datasets)
        for data in acc:
            print(f'Evaluation accurracy of dataset {data} using pre-trained weights is {acc[data]}')
        
         # train only last layer with ImgNet initialized weights 
        print("\nUsing model " + model_name + " pre-trained and training only the last layer")
        print("Starting training")
        fe_model, fe_optimizer_ft, fe_exp_lr_scheduler = initialize_model(model_name, len(class_names), train_last_layer_only=True, use_pretrained=True)
        fe_model = fe_model.to(device)
        fe_criterion = nn.CrossEntropyLoss()
        best_fe_model, fe_train_lss, fe_val_lss, fe_train_acc,fe_val_acc = train_model(fe_model, fe_criterion, fe_optimizer_ft, fe_exp_lr_scheduler,num_epochs=epochs)
        fe_acc = evaluate_model(best_fe_model, eval_datasets)
        for data in fe_acc:
            print(f'Evaluation accurracy of dataset {data} using feature extraction is {fe_acc[data]}')

        # train scratch model
        print("\nUsing model " + model_name + " not pre-trained")
        scratch_model, scratch_optimizer_ft, scratch_exp_lr_scheduler = initialize_model(model_name, len(class_names), train_last_layer_only=False, use_pretrained=False)
        scratch_model = scratch_model.to(device)
        scratch_criterion = nn.CrossEntropyLoss()
        best_scratch_model,scratch_train_lss, scratch_val_lss, scratch_train_acc,scratch_val_acc = train_model(scratch_model, scratch_criterion, scratch_optimizer_ft, scratch_exp_lr_scheduler, num_epochs=epochs)
        scratch_acc = evaluate_model(best_scratch_model, eval_datasets)
        for data in scratch_acc:
            print(f'Evaluation accurracy of dataset {data} using the scratch model is {scratch_acc[data]}')

            
        # draw performance graphs
        plot_graphs(model_name, train_lss, val_lss, train_acc,val_acc,scratch_train_lss, scratch_val_lss, scratch_train_acc,scratch_val_acc,fe_train_lss, fe_val_lss, fe_train_acc,fe_val_acc, epochs)
        
    

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    train_lss, val_lss, train_acc,val_acc = [], [], [], []
    since = time.time() #starting time

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            #add to lists
            if phase == 'train':
                train_lss.append(float(epoch_loss))
                train_acc.append(float(epoch_acc))
            else:
                val_lss.append(float(epoch_loss))
                val_acc.append(float(epoch_acc))

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_lss, val_lss, train_acc, val_acc

In [19]:
def evaluate_model(model, dataset):
    epoch_acc = {}
    for dataset in eval_datasets:
        print(f'Starting evaluation of set {dataset} consisting of {eval_dataset_sizes[dataset]} images')
        #print(eval_datasets[dataset])
        eval_acc = []
        model.eval()            
        running_corrects = 0
        since = time.time() #starting time
        for inputs, labels in eval_dataloaders[dataset]:
            inputs = inputs.to(device)
            if(dataset == 'live'): # workaround for wrong label. fake = 0, live = 1. As there is no fake data live got index 0 instead of 1
                labels = labels + 1
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            #loss = criterion(outputs, labels)

            # statistics
            # count correct predictions
            running_corrects += torch.sum(preds == labels.data)

        print(f' running corrects {running_corrects} / dataset size {eval_dataset_sizes[dataset]}')
        epoch_acc[dataset] = running_corrects.double() / eval_dataset_sizes[dataset]

        time_elapsed = time.time() - since
        print(f'Evaluation of set {dataset} complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Evaluation Acc: {epoch_acc[dataset]:4f}')

    return epoch_acc

In [ ]:
def plot_graphs(model_name, train_lss, val_lss, train_acc,val_acc,scratch_train_lss, scratch_val_lss, scratch_train_acc,scratch_val_acc,fe_train_lss, fe_val_lss, fe_train_acc,fe_val_acc, epochs):
    # PT-AL: pre-trained weights, further training all layers
    # PT-LL: pre-trained weights, training only last layer
    # Scratch: Random initialized weights, training all layers
    
    plt.title("Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Validation Accuracy")
    plt.plot(range(1,epochs+1),val_acc,label="PT-AL")
    plt.plot(range(1,epochs+1),scratch_val_acc,label="Scratch")
    plt.plot(range(1,epochs+1),fe_val_acc,label="PT-LL")
    plt.ylim((0,1.))
    #plt.xticks(np.arange(1, epochs_amount+1, 5.0))
    plt.legend()
    plt.savefig(model_name + "_all_modes_acc.png")
    plt.show()

    plt.title("Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Validation Loss")
    plt.plot(range(1,epochs+1),val_lss,label="Pretrained")
    plt.plot(range(1,epochs+1),scratch_val_lss,label="Scratch")
    plt.plot(range(1,epochs+1),fe_val_lss,label="Featureextraction")
    plt.ylim((0,1.))
    #plt.xticks(np.arange(1, epochs_amount+1, 5.0))
    plt.legend()
    plt.savefig(model_name + "_all_modes_loss.png")
    plt.show()
    
    
    plt.plot(range(1,epochs+1), train_lss, 'g', label='Training loss')
    plt.plot(range(1,epochs+1), val_lss, 'b', label='Validation loss')
    plt.ylim((0,1.))
    plt.title('Training and Validation loss PT-AL')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    #plt.xticks(np.arange(1, epochs_amount+1, 5.0))
    plt.ylim((0,1.))
    plt.legend()
    plt.savefig(model_name + "_PT_AL_loss.png")
    plt.show()

    plt.plot(range(1,epochs+1), train_acc, 'g', label='Training acc.')
    plt.plot(range(1,epochs+1), val_acc, 'b', label='Validation acc.')
    plt.ylim((0,1.))
    plt.title('Training and Validation accuracy PT_AL')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    #plt.xticks(np.arange(1, epochs_amount+1, 5.0))
    plt.legend()
    plt.savefig(model_name + "_PT_AL_acc.png")
    plt.show()
    
    plt.plot(range(1,epochs+1), scratch_train_lss, 'g', label='Training loss')
    plt.plot(range(1,epochs+1), scratch_val_lss, 'b', label='Validation loss')
    plt.ylim((0,1.))
    plt.title('Training and Validation loss from scratch')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    #plt.xticks(np.arange(1, epochs_amount+1, 5.0))
    plt.savefig(model_name + "_scratch_loss.png")
    plt.show()

    plt.plot(range(1,epochs+1), scratch_train_acc, 'g', label='Training acc.')
    plt.plot(range(1,epochs+1), scratch_val_acc, 'b', label='Validation acc.')
    plt.ylim((0,1.))
    plt.title('Training and Validation accuracy from scratch')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    #plt.xticks(np.arange(1, epochs_amount+1, 5.0))
    plt.savefig(model_name + "_scratch_acc.png")
    plt.show()
    
    plt.plot(range(1,epochs+1), fe_train_lss, 'g', label='Training loss')
    plt.plot(range(1,epochs+1), fe_val_lss, 'b', label='Validation loss')
    plt.ylim((0,1.))
    plt.title('Training and Validation loss PT_LL')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    #plt.xticks(np.arange(1, epochs_amount+1, 5.0))
    plt.savefig(model_name + "_PT_LL_loss.png")
    plt.show()

    plt.plot(range(1,epochs+1), fe_train_acc, 'g', label='Training acc.')
    plt.plot(range(1,epochs+1), fe_val_acc, 'b', label='Validation acc.')
    plt.ylim((0,1.))
    plt.title('Training and Validation accuracy PT_LL')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    #plt.xticks(np.arange(1, epochs_amount+1, 5.0))
    plt.savefig(model_name + "_PT_LL_acc.png")
    plt.show()



In [ ]:
models_to_train = ["resnet", "alexnet", "vgg16", "squeezenet", "densenet"]
train_all(models_to_train, 2)